#Install Libraries

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 649.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


# Load and Process Data

In [ ]:
from gensim.models import KeyedVectors

def load_top_words(path, top_k=100000):
    model = KeyedVectors.load_word2vec_format(path, limit=top_k)
    return model

In [ ]:
def load_lexicon(path, src_model, tgt_model):
    pairs = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            src, tgt = line.strip().split()
            if src in src_model and tgt in tgt_model:
                pairs.append((src, tgt))
    return pairs

In [ ]:
import numpy as np

def get_embedding_matrices(pairs, src_model, tgt_model):
    src_matrix = []
    tgt_matrix = []
    for src, tgt in pairs:
        src_matrix.append(src_model[src])
        tgt_matrix.append(tgt_model[tgt])
    return np.array(src_matrix).T, np.array(tgt_matrix).T

In [ ]:
def procrustes(X, Y):
    # Compute optimal orthogonal mapping W
    U, _, Vt = np.linalg.svd(Y @ X.T)
    W = U @ Vt
    return W

In [ ]:
def apply_mapping(src_model, W):
    mapped_vectors = {word: W @ src_model[word] for word in src_model.key_to_index}
    return mapped_vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def translate_words(src_words, mapped_src_vectors, tgt_model, top_k=5):
    results = {}
    tgt_matrix = np.array([tgt_model[word] for word in tgt_model.key_to_index])
    tgt_words = list(tgt_model.key_to_index.keys())

    for word in src_words:
        if word in mapped_src_vectors:
            vec = mapped_src_vectors[word].reshape(1, -1)
            sims = cosine_similarity(vec, tgt_matrix).flatten()
            top_k_indices = np.argsort(-sims)[:top_k]
            results[word] = [tgt_words[i] for i in top_k_indices]
    return results

In [ ]:
def precision_at_k(results, gold_dict, k=1):
    correct = 0
    total = 0
    for word in results:
        if word in gold_dict:
            total += 1
            correct += int(gold_dict[word] in results[word][:k])
    return correct / total

#Load Text Embeddings

In [ ]:
en_model = load_top_words('/content/drive/MyDrive/cc.en.300.vec')
hi_model = load_top_words('/content/drive/MyDrive/cc.hi.300.vec')

In [ ]:
bilingual_lexicon = load_lexicon('en-hi.txt', en_model, hi_model)

In [ ]:
X, Y = get_embedding_matrices(bilingual_lexicon, en_model, hi_model)

#Perform Procrustes Method to find similarity

In [ ]:
from scipy.spatial import procrustes

def scipy_procrustes(X, Y):
    """Aligns X to Y using Scipy's built-in Procrustes method"""
    X_aligned, Y_aligned, disparity = procrustes(X.T, Y.T)  # Transpose back to original format
    return X_aligned.T, disparity  # Return transposed back
X_procrustes, disparity = scipy_procrustes(X, Y)

In [ ]:
W = procrustes(X, Y)

In [ ]:
mapped_en_vectors = apply_mapping(en_model, W)

In [ ]:
test_words = ["cat", "house", "river"]
translations = translate_words(test_words, mapped_en_vectors, hi_model, top_k=5)

for word, trans in translations.items():
    print(f"{word} -> {trans}")

cat -> ['बिल्ली', 'कुत्ता', 'कुत्ते', 'पालतू', 'कुत्तों']
house -> ['मकान', 'फ़्लैट', 'घर', 'कमरे', 'हाउस']
river -> ['नदी', 'किनारे', 'गंगा', 'चनाब', 'नहर']


#Results

In [ ]:
gold_dict = {src: tgt for src, tgt in bilingual_lexicon}

# Compute Precision@1 and Precision@5
p1 = precision_at_k(translations, gold_dict, k=1)
p5 = precision_at_k(translations, gold_dict, k=5)

print(f"Precision@1: {p1:.4f}")
print(f"Precision@5: {p5:.4f}")

Precision@1: 0.6667
Precision@5: 1.0000


In [ ]:
sizes = [5000, 10000, 20000]
for size in sizes:
    small_lexicon = bilingual_lexicon[:size]
    X_small, Y_small = get_embedding_matrices(small_lexicon, en_model, hi_model)
    W_small = procrustes(X_small, Y_small)
    mapped_small = apply_mapping(en_model, W_small)
    translations_small = translate_words(test_words, mapped_small, hi_model, top_k=5)
    p1_small = precision_at_k(translations_small, gold_dict, k=1)
    p5_small = precision_at_k(translations_small, gold_dict, k=5)
    print(f"Dictionary Size: {size}, Precision@1: {p1_small:.4f}, Precision@5: {p5_small:.4f}")


Dictionary Size: 5000, Precision@1: 0.6667, Precision@5: 1.0000
Dictionary Size: 10000, Precision@1: 0.6667, Precision@5: 0.6667
Dictionary Size: 20000, Precision@1: 0.6667, Precision@5: 1.0000
